# Asymmetric Random Restart Strategy

The key insight from web search is that top scores below 69 are achieved by:
1. ASYMMETRIC solutions - exploit irregular gaps between trees
2. Symmetry-breaking tricks - force asymmetric configurations early

This notebook generates random asymmetric starting configurations and runs SA optimization to find different basins.

In [1]:
import math
import numpy as np
import pandas as pd
import time
import os
from numba import njit
from multiprocessing import Pool, cpu_count

print(f"CPU count: {cpu_count()}")

CPU count: 26


In [2]:
# Tree shape constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H
MAX_OVERLAP_DIST_SQ = 4.0

@njit(cache=True)
def rotate_point(x, y, cos_a, sin_a):
    return x * cos_a - y * sin_a, x * sin_a + y * cos_a

@njit(cache=True)
def get_tree_vertices(cx, cy, angle_deg):
    angle_rad = angle_deg * math.pi / 180.0
    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)
    vertices = np.empty((15, 2), dtype=np.float64)
    pts = np.array([
        [0.0, TIP_Y], [TOP_W / 2.0, TIER_1_Y], [TOP_W / 4.0, TIER_1_Y],
        [MID_W / 2.0, TIER_2_Y], [MID_W / 4.0, TIER_2_Y], [BASE_W / 2.0, BASE_Y],
        [TRUNK_W / 2.0, BASE_Y], [TRUNK_W / 2.0, TRUNK_BOTTOM_Y],
        [-TRUNK_W / 2.0, TRUNK_BOTTOM_Y], [-TRUNK_W / 2.0, BASE_Y],
        [-BASE_W / 2.0, BASE_Y], [-MID_W / 4.0, TIER_2_Y], [-MID_W / 2.0, TIER_2_Y],
        [-TOP_W / 4.0, TIER_1_Y], [-TOP_W / 2.0, TIER_1_Y],
    ], dtype=np.float64)
    for i in range(15):
        rx, ry = rotate_point(pts[i, 0], pts[i, 1], cos_a, sin_a)
        vertices[i, 0] = rx + cx
        vertices[i, 1] = ry + cy
    return vertices

In [3]:
@njit(cache=True)
def polygon_bounds(vertices):
    min_x = min_y = 1e10
    max_x = max_y = -1e10
    for i in range(vertices.shape[0]):
        x, y = vertices[i, 0], vertices[i, 1]
        if x < min_x: min_x = x
        if x > max_x: max_x = x
        if y < min_y: min_y = y
        if y > max_y: max_y = y
    return min_x, min_y, max_x, max_y

@njit(cache=True)
def point_in_polygon(px, py, vertices):
    n = vertices.shape[0]
    inside = False
    j = n - 1
    for i in range(n):
        xi, yi = vertices[i, 0], vertices[i, 1]
        xj, yj = vertices[j, 0], vertices[j, 1]
        if ((yi > py) != (yj > py)) and (px < (xj - xi) * (py - yi) / (yj - yi) + xi):
            inside = not inside
        j = i
    return inside

@njit(cache=True)
def segments_intersect(p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y):
    dax, day = p2x - p1x, p2y - p1y
    dbx, dby = p4x - p3x, p4y - p3y
    d1x, d1y = p1x - p3x, p1y - p3y
    d2x, d2y = p2x - p3x, p2y - p3y
    cross_b1 = dbx * d1y - dby * d1x
    cross_b2 = dbx * d2y - dby * d2x
    if cross_b1 * cross_b2 > 0: return False
    d3x, d3y = p3x - p1x, p3y - p1y
    d4x, d4y = p4x - p1x, p4y - p1y
    cross_a1 = dax * d3y - day * d3x
    cross_a2 = dax * d4y - day * d4x
    if cross_a1 * cross_a2 > 0: return False
    return True

In [4]:
@njit(cache=True)
def polygons_overlap(verts1, verts2, cx1, cy1, cx2, cy2):
    dx, dy = cx2 - cx1, cy2 - cy1
    if dx * dx + dy * dy > MAX_OVERLAP_DIST_SQ: return False
    min_x1, min_y1, max_x1, max_y1 = polygon_bounds(verts1)
    min_x2, min_y2, max_x2, max_y2 = polygon_bounds(verts2)
    if max_x1 < min_x2 or max_x2 < min_x1 or max_y1 < min_y2 or max_y2 < min_y1: return False
    for i in range(verts1.shape[0]):
        if point_in_polygon(verts1[i, 0], verts1[i, 1], verts2): return True
    for i in range(verts2.shape[0]):
        if point_in_polygon(verts2[i, 0], verts2[i, 1], verts1): return True
    n1, n2 = verts1.shape[0], verts2.shape[0]
    for i in range(n1):
        j = (i + 1) % n1
        for k in range(n2):
            m = (k + 1) % n2
            if segments_intersect(verts1[i,0], verts1[i,1], verts1[j,0], verts1[j,1],
                                  verts2[k,0], verts2[k,1], verts2[m,0], verts2[m,1]):
                return True
    return False

@njit(cache=True)
def has_any_overlap(all_vertices, xs, ys):
    n = len(all_vertices)
    for i in range(n):
        for j in range(i + 1, n):
            if polygons_overlap(all_vertices[i], all_vertices[j], xs[i], ys[i], xs[j], ys[j]):
                return True
    return False

@njit(cache=True)
def get_side_length(all_vertices):
    min_x = min_y = 1e10
    max_x = max_y = -1e10
    for verts in all_vertices:
        for i in range(verts.shape[0]):
            x, y = verts[i, 0], verts[i, 1]
            if x < min_x: min_x = x
            if x > max_x: max_x = x
            if y < min_y: min_y = y
            if y > max_y: max_y = y
    return max(max_x - min_x, max_y - min_y)

@njit(cache=True)
def calculate_score(all_vertices):
    side = get_side_length(all_vertices)
    return side * side / len(all_vertices)

In [5]:
# Generate random asymmetric configuration
@njit(cache=True)
def generate_random_config(n, seed, box_size=5.0):
    """Generate random positions and angles for n trees."""
    np.random.seed(seed)
    xs = np.random.uniform(-box_size, box_size, n)
    ys = np.random.uniform(-box_size, box_size, n)
    degs = np.random.uniform(0, 360, n)  # Random angles, not just 0/180
    return xs, ys, degs

@njit(cache=True)
def check_valid_config(xs, ys, degs):
    """Check if configuration has no overlaps."""
    n = len(xs)
    vertices = [get_tree_vertices(xs[i], ys[i], degs[i]) for i in range(n)]
    return not has_any_overlap(vertices, xs, ys)

In [6]:
# SA optimization for a single configuration
@njit(cache=True)
def sa_optimize_config(xs_init, ys_init, degs_init, Tmax, Tmin, nsteps, nsteps_per_T, 
                       position_delta, angle_delta, random_seed):
    """SA optimization starting from given configuration."""
    np.random.seed(random_seed)
    n = len(xs_init)
    
    xs = xs_init.copy()
    ys = ys_init.copy()
    degs = degs_init.copy()
    
    vertices = [get_tree_vertices(xs[i], ys[i], degs[i]) for i in range(n)]
    current_score = calculate_score(vertices)
    
    best_score = current_score
    best_xs = xs.copy()
    best_ys = ys.copy()
    best_degs = degs.copy()
    
    T = Tmax
    Tfactor = -math.log(Tmax / Tmin) if Tmax > Tmin else 0
    
    for step in range(nsteps):
        for _ in range(nsteps_per_T):
            i = np.random.randint(0, n)
            old_x, old_y, old_deg = xs[i], ys[i], degs[i]
            
            xs[i] += (np.random.random() * 2.0 - 1.0) * position_delta
            ys[i] += (np.random.random() * 2.0 - 1.0) * position_delta
            degs[i] = (degs[i] + (np.random.random() * 2.0 - 1.0) * angle_delta) % 360.0
            
            # Check for overlaps with this tree
            new_vert = get_tree_vertices(xs[i], ys[i], degs[i])
            has_overlap = False
            for j in range(n):
                if i != j:
                    other_vert = get_tree_vertices(xs[j], ys[j], degs[j])
                    if polygons_overlap(new_vert, other_vert, xs[i], ys[i], xs[j], ys[j]):
                        has_overlap = True
                        break
            
            if has_overlap:
                xs[i], ys[i], degs[i] = old_x, old_y, old_deg
                continue
            
            new_vertices = [get_tree_vertices(xs[k], ys[k], degs[k]) for k in range(n)]
            new_score = calculate_score(new_vertices)
            delta = new_score - current_score
            
            accept = delta < 0 or (T > 1e-10 and np.random.random() < math.exp(-delta / T))
            
            if accept:
                current_score = new_score
                if new_score < best_score:
                    best_score = new_score
                    best_xs = xs.copy()
                    best_ys = ys.copy()
                    best_degs = degs.copy()
            else:
                xs[i], ys[i], degs[i] = old_x, old_y, old_deg
        
        T = Tmax * math.exp(Tfactor * (step + 1) / nsteps)
    
    return best_score, best_xs, best_ys, best_degs

In [7]:
# Load baseline for comparison
def load_baseline():
    baseline_path = '/home/code/external_data/saspav_latest/santa-2025.csv'
    df = pd.read_csv(baseline_path)
    for col in ['x', 'y', 'deg']:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.replace('s', '').astype(float)
    
    baseline = {}
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        group = df[df["id"].str.startswith(prefix)].sort_values("id")
        if len(group) == n:
            baseline[n] = {
                'xs': group['x'].values.astype(np.float64),
                'ys': group['y'].values.astype(np.float64),
                'degs': group['deg'].values.astype(np.float64)
            }
    return baseline

baseline = load_baseline()
print(f"Loaded baseline with {len(baseline)} configurations")

# Calculate baseline scores
baseline_scores = {}
for n in range(1, 201):
    if n in baseline:
        vertices = [get_tree_vertices(baseline[n]['xs'][i], baseline[n]['ys'][i], baseline[n]['degs'][i]) for i in range(n)]
        baseline_scores[n] = calculate_score(vertices)

baseline_total = sum(baseline_scores.values())
print(f"Baseline total score: {baseline_total:.6f}")

Loaded baseline with 200 configurations


Baseline total score: 70.659958


In [8]:
# Warm up numba
print("Compiling numba functions...")
t0 = time.time()
dummy_xs = np.array([0.0, 1.0], dtype=np.float64)
dummy_ys = np.array([0.0, 0.0], dtype=np.float64)
dummy_degs = np.array([0.0, 180.0], dtype=np.float64)
_ = sa_optimize_config(dummy_xs, dummy_ys, dummy_degs, 0.01, 0.0001, 2, 100, 0.1, 10.0, 42)
print(f"Compilation done in {time.time() - t0:.1f}s")

Compiling numba functions...


Compilation done in 1.4s


In [9]:
# Try random restart strategy for a few N values
# Focus on small N values which have worst packing efficiency

sa_params = {
    'Tmax': 0.1,
    'Tmin': 0.00001,
    'nsteps': 20,
    'nsteps_per_T': 5000,  # 100k moves total
    'position_delta': 0.05,
    'angle_delta': 10.0,
}

test_n_values = [2, 3, 4, 5, 6, 7, 8, 9, 10]
num_random_starts = 20

print(f"Testing random restart strategy on N={test_n_values}")
print(f"Random starts per N: {num_random_starts}")
print(f"SA moves per start: {sa_params['nsteps'] * sa_params['nsteps_per_T']:,}")

improvements = {}

for n in test_n_values:
    baseline_score = baseline_scores[n]
    best_score = baseline_score
    best_config = None
    
    print(f"\nN={n}: baseline={baseline_score:.6f}")
    
    for seed in range(num_random_starts):
        # Generate random starting configuration
        xs, ys, degs = generate_random_config(n, seed * 1000 + n, box_size=2.0)
        
        # Check if valid (no overlaps)
        if not check_valid_config(xs, ys, degs):
            # Try to find a valid config by spreading out
            xs, ys, degs = generate_random_config(n, seed * 1000 + n, box_size=5.0)
            if not check_valid_config(xs, ys, degs):
                continue
        
        # Run SA optimization
        score, opt_xs, opt_ys, opt_degs = sa_optimize_config(
            xs, ys, degs,
            sa_params['Tmax'], sa_params['Tmin'],
            sa_params['nsteps'], sa_params['nsteps_per_T'],
            sa_params['position_delta'], sa_params['angle_delta'],
            seed
        )
        
        if score < best_score:
            best_score = score
            best_config = (opt_xs.copy(), opt_ys.copy(), opt_degs.copy())
            print(f"  Seed {seed}: Found better score {score:.6f} (improvement: {baseline_score - score:.6f})")
    
    if best_config is not None and best_score < baseline_score:
        improvements[n] = {
            'score': best_score,
            'xs': best_config[0],
            'ys': best_config[1],
            'degs': best_config[2],
            'improvement': baseline_score - best_score
        }
        print(f"  BEST: {best_score:.6f} (improvement: {baseline_score - best_score:.6f})")
    else:
        print(f"  No improvement found")

print(f"\nTotal improvements found: {len(improvements)}")

Testing random restart strategy on N=[2, 3, 4, 5, 6, 7, 8, 9, 10]
Random starts per N: 20
SA moves per start: 100,000

N=2: baseline=0.450779


  No improvement found

N=3: baseline=0.434745


  No improvement found

N=4: baseline=0.416545


  No improvement found

N=5: baseline=0.416850


  No improvement found

N=6: baseline=0.399610


  No improvement found

N=7: baseline=0.399897


  No improvement found

N=8: baseline=0.385407


  No improvement found

N=9: baseline=0.387415


  No improvement found

N=10: baseline=0.376630


  No improvement found

Total improvements found: 0


In [10]:
# Try with larger N values and more aggressive SA
# The baseline might have more room for improvement on larger N

sa_params_aggressive = {
    'Tmax': 1.0,  # Higher starting temperature
    'Tmin': 0.000001,
    'nsteps': 30,
    'nsteps_per_T': 10000,  # 300k moves total
    'position_delta': 0.1,
    'angle_delta': 20.0,
}

test_n_values_large = [20, 30, 40, 50, 60, 70, 80, 90, 100]
num_random_starts = 10

print(f"Testing random restart strategy on larger N={test_n_values_large}")
print(f"Random starts per N: {num_random_starts}")
print(f"SA moves per start: {sa_params_aggressive['nsteps'] * sa_params_aggressive['nsteps_per_T']:,}")

for n in test_n_values_large:
    baseline_score = baseline_scores[n]
    best_score = baseline_score
    
    print(f"\nN={n}: baseline={baseline_score:.6f}")
    
    for seed in range(num_random_starts):
        # Generate random starting configuration with larger box
        box_size = math.sqrt(n) * 1.5  # Scale box with sqrt(n)
        xs, ys, degs = generate_random_config(n, seed * 1000 + n, box_size=box_size)
        
        # Check if valid
        if not check_valid_config(xs, ys, degs):
            # Try larger box
            xs, ys, degs = generate_random_config(n, seed * 1000 + n, box_size=box_size * 2)
            if not check_valid_config(xs, ys, degs):
                continue
        
        # Run SA optimization
        score, opt_xs, opt_ys, opt_degs = sa_optimize_config(
            xs, ys, degs,
            sa_params_aggressive['Tmax'], sa_params_aggressive['Tmin'],
            sa_params_aggressive['nsteps'], sa_params_aggressive['nsteps_per_T'],
            sa_params_aggressive['position_delta'], sa_params_aggressive['angle_delta'],
            seed
        )
        
        if score < best_score:
            best_score = score
            print(f"  Seed {seed}: Found better score {score:.6f} (improvement: {baseline_score - score:.6f})")
    
    if best_score < baseline_score:
        print(f"  BEST: {best_score:.6f} (improvement: {baseline_score - best_score:.6f})")
    else:
        print(f"  No improvement found")

Testing random restart strategy on larger N=[20, 30, 40, 50, 60, 70, 80, 90, 100]
Random starts per N: 10
SA moves per start: 300,000

N=20: baseline=0.376057


  No improvement found

N=30: baseline=0.360883


  No improvement found

N=40: baseline=0.362148


  No improvement found

N=50: baseline=0.360753


  No improvement found

N=60: baseline=0.357258
  No improvement found

N=70: baseline=0.349513


  No improvement found

N=80: baseline=0.344880
  No improvement found

N=90: baseline=0.343155
  No improvement found

N=100: baseline=0.345531
  No improvement found


In [ ]:
# Try very long SA optimization starting from baseline
# This is different from random restart - we start from the known good solution

sa_params_long = {
    'Tmax': 0.001,  # Low temperature for fine-tuning
    'Tmin': 0.0000001,
    'nsteps': 50,
    'nsteps_per_T': 20000,  # 1M moves total
    'position_delta': 0.001,  # Very small moves
    'angle_delta': 0.5,
}

test_n_values = [50, 100, 150, 200]

print(f"Testing long SA from baseline on N={test_n_values}")
print(f"SA moves per N: {sa_params_long['nsteps'] * sa_params_long['nsteps_per_T']:,}")

for n in test_n_values:
    baseline_score = baseline_scores[n]
    xs = baseline[n]['xs'].copy()
    ys = baseline[n]['ys'].copy()
    degs = baseline[n]['degs'].copy()
    
    print(f"\nN={n}: baseline={baseline_score:.9f}")
    
    t0 = time.time()
    score, opt_xs, opt_ys, opt_degs = sa_optimize_config(
        xs, ys, degs,
        sa_params_long['Tmax'], sa_params_long['Tmin'],
        sa_params_long['nsteps'], sa_params_long['nsteps_per_T'],
        sa_params_long['position_delta'], sa_params_long['angle_delta'],
        42
    )
    elapsed = time.time() - t0
    
    diff = score - baseline_score
    if diff < -1e-9:
        print(f"  IMPROVED: {score:.9f} (improvement: {-diff:.9f}) [{elapsed:.1f}s]")
    else:
        print(f"  No improvement: {score:.9f} [{elapsed:.1f}s]")

In [ ]:
# Calculate total improvement
if improvements:
    total_improvement = sum(imp['improvement'] for imp in improvements.values())
    print(f"Total improvement from random restarts: {total_improvement:.6f}")
    
    new_total = baseline_total - total_improvement
    print(f"New total score: {new_total:.6f}")
    print(f"Baseline total: {baseline_total:.6f}")
else:
    print("No improvements found")
    new_total = baseline_total

In [ ]:
# Save submission with improvements
print("\nSaving submission...")
os.makedirs('/home/submission', exist_ok=True)

rows = []
for n in range(1, 201):
    if n in improvements:
        config = improvements[n]
        xs, ys, degs = config['xs'], config['ys'], config['degs']
    else:
        xs = baseline[n]['xs']
        ys = baseline[n]['ys']
        degs = baseline[n]['degs']
    
    for i in range(n):
        rows.append({
            'id': f'{n:03d}_{i:03d}',
            'x': xs[i],
            'y': ys[i],
            'angle': degs[i]
        })

df = pd.DataFrame(rows)
df.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved to /home/submission/submission.csv")
print(f"Total rows: {len(df)}")
print(f"\nFinal score: {new_total:.9f}")